In [ ]:
%matplotlib inline

In [ ]:
# cell 0
import glob, pickle, csv
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D               # noqa: F401 (side-effect)
from matplotlib.animation import FuncAnimation
import ipywidgets as widgets
from IPython.display import display, HTML


In [ ]:
# cell 1 ── locate & load every snapshot  (FIXED)
run = Path("/Users/c3495249/Coding/growth_based_optimization/gbo/demo_1")  # ← your output dir
assert run.is_dir(), f"{run} does not exist"

snap_paths = sorted(run.glob("snapshot_*.pkl"))
assert snap_paths, f"No snapshot_XXXXX.pkl files found in {run}"

snapshots = []
for p in snap_paths:
    with p.open("rb") as fh:
        snapshots.append(pickle.load(fh))

print(f"{len(snapshots)} snapshots loaded "
      f"(iters {snapshots[0]['iter']} … {snapshots[-1]['iter']})")


In [ ]:
# cell 2 ── small convenience: convert a VascularTree into numpy arrays for plotting
def tree_to_arrays(tree):
    G = tree.graph
    # node coordinates & radii
    xyz = np.array([G.nodes[n]["data"].pos for n in G.nodes])
    r   = np.array([G.nodes[n]["data"].radius for n in G.nodes])
    # every edge as two 3-D points  (for fast Line3DCollection later if you wish)
    edges = []
    for u, v in G.edges:
        edges.append((G.nodes[u]["data"].pos, G.nodes[v]["data"].pos))
    return xyz, r, np.array(edges)        # xyz, radii, (E,2,3)


In [ ]:
# cell 3 ── set up matplotlib figure (tree on the left, supply-map slice on the right)
fig = plt.figure(figsize=(12, 5))
ax_tree   = fig.add_subplot(1, 2, 1, projection='3d')
ax_supply = fig.add_subplot(1, 2, 2)
ax_tree.set_title("vascular tree")
ax_supply.set_title("supply map (mid-Z slice)")
plt.tight_layout()


In [ ]:
# cell 4 ── interactive slider to scrub frames
slider = widgets.IntSlider(
    value=0,
    min=0,
    max=len(snapshots)-1,
    step=1,
    description='iter:',
    continuous_update=False,
)
display(slider)


In [ ]:
# cell 5
def draw_frame(idx):
    snap = snapshots[idx]
    tree   = snap["tree"]
    tissue = snap["tissue"]
    
    # ---------- left: tree --------------------------------------------------
    ax_tree.cla()
    xyz, r, edges = tree_to_arrays(tree)
    # plot edges
    for seg in edges:
        ax_tree.plot(*seg.T, color="tab:blue", linewidth=1.0)
    # plot terminals in orange
    term_ids = tree.terminals()
    term_xyz = xyz[term_ids]
    ax_tree.scatter(*term_xyz.T, s=10, color="tab:orange")
    ax_tree.set_xlim(-30, 30); ax_tree.set_ylim(-30, 30); ax_tree.set_zlim(-30, 30)
    ax_tree.set_xlabel("x [mm]"); ax_tree.set_ylabel("y [mm]"); ax_tree.set_zlabel("z [mm]")
    ax_tree.set_title(f"Tree – iter {snap['iter']}  (|tips|={len(term_ids)})")

    # ---------- right: supply map -------------------------------------------
    # supplied  : voxels with ownership >= 0
    # unsupplied: ownership == -1     (inside Ω yet not covered)
    own = tissue.ownership
    mid_z = own.shape[0] // 2
    slc  = own[mid_z]
    ax_supply.cla()
    ax_supply.imshow(slc >= 0, cmap="Greys", origin="lower")
    ax_supply.set_title("(supplied = white)\nZ-slice {:.0f} mm".format(
        tissue.origin[2] + (mid_z+0.5)*tissue.spacing))
    ax_supply.set_xticks([]); ax_supply.set_yticks([])

    fig.canvas.draw_idle()


In [ ]:
# cell 6 ── update on slider drag
slider.observe(lambda ch: draw_frame(ch["new"]), names="value")
# draw first frame
draw_frame(0)


In [ ]:
# cell 7 ── automatic animation (comment out if you prefer manual slider only)
def _update(frame):
    slider.value = frame
    return []           # FuncAnimation needs something iterable

ani = FuncAnimation(fig, _update, frames=len(snapshots), interval=400, blit=True)
HTML(ani.to_jshtml())    # inline in Jupyter


In [ ]:
# cell 8 ── one-time loss curve
loss_file = Path("loss_history.csv")
if loss_file.exists():
    iters, losses = [], []
    with loss_file.open() as fh:
        reader = csv.DictReader(fh)
        for row in reader:
            iters.append(int(row["iter"]))
            # if you stored more components add them here; we stored L_total only
            losses.append(float(row["L_total"]))
    plt.figure()
    plt.semilogy(iters, losses)
    plt.xlabel("iteration")
    plt.ylabel("total loss L")
    plt.title("Loss history")
    plt.grid(True, which='both')
else:
    print("loss_history.csv not found – did you enable snapshot / loss logging?")


In [ ]:
# ----- 1. plot loss curve -------------------------------------------
loss = pd.read_csv(run / "loss_history.csv")
fig, ax = plt.subplots()
ax.plot(loss["iter"], loss["L_total"])
ax.set(xlabel="iteration", ylabel="Total loss L")

In [ ]:
# ----- 2. animate supplied voxels -----------------------------------
from matplotlib.animation import FuncAnimation
snaps = sorted(run.glob("tissue_*.npz"))

fig2, ax2 = plt.subplots()
im = ax2.imshow(np.zeros((64,64)))       # dummy init

def update(i):
    data = np.load(snaps[i])["ownership"]
    slice_xy = data[:, :, data.shape[2]//2]   # mid-Z slice
    im.set_data(slice_xy)
    ax2.set_title(f"iter {i}")              # 5 if snapshot_every == 5
    return (im,)

ani = FuncAnimation(fig2, update, frames=len(snaps), interval=200)

In [ ]:
ani.save("supply_evolution.gif", dpi=80)


In [ ]:
# ─── nodes & edges of a given iter ──────────────────────────────
it = 11
nodes = pd.read_csv(run_dir / f"nodes_{it:05d}.csv")
edges = pd.read_csv(run_dir / f"edges_{it:05d}.csv")

# ─── supply map ─────────────────────────────────────────────────
data   = np.load(run_dir / f"tissue_{it:05d}.npz")
owner  = data["ownership"]         # int32
demand = data["demand"]            # float32

# ─── metadata & losses ──────────────────────────────────────────
with open(run_dir / f"meta_{it:05d}.json") as fh:
    meta = json.load(fh)
print("iteration", meta["iter"], "losses", meta["losses"])

# ─── quick visual: supplied vs unsupplied voxels ────────────────
plt.imshow(owner[owner.shape[0]//2,:,:], origin="lower")
plt.title(f"ownership map, z-midplane, iter {it}")
plt.colorbar(); plt.show()